In [1]:
import ase
import ase.io
from ase.io import xsf
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/Users/mohammad/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
with open(os.path.join('data', 'structures_df2_calc_with_bands_be.json')) as fhandle:
        df2 = json.load(fhandle)
with open(os.path.join('data', 'structures_rvv10_calc_with_bands_be.json')) as fhandle:
        rvv10 = json.load(fhandle)
with open(os.path.join('data', 'structures_revpbe_calc_with_bands_be.json')) as fhandle:
        revpbe = json.load(fhandle)

In [7]:
    binding_energies_raw = {}
    for uuid_key in set(df2).union(rvv10):
        subdict = {}
        df2_binding_energy = df2.get(uuid_key, {}).get('binding_energy', {})
        if df2_binding_energy:
            subdict['df2'] = df2_binding_energy
        if subdict:
            binding_energies_raw[uuid_key] = subdict
    
    print("Found {} entries, these should be set D of the Mounet paper, maybe there is one structure that has been 'removed' by hand".format(
        len(binding_energies_raw)))

Found 2492 entries, these should be set D of the Mounet paper, maybe there is one structure that has been 'removed' by hand


In [8]:
import os

path = 'data/structures/structures_3D_xsf/'

folder = os.fsencode(path)

filenames = []
structures = []
uiids = []

strucutres_not_found=0
for uiid in binding_energies_raw.keys():
    
    try:
        filename = uiid + '.xsf'
    
        with open(path + filename) as f:
            structure = ase.io.xsf.read_xsf(f, index=- 1, read_data=False)
        
            structures.append(structure)
            uiids.append(uiid)
    except FileNotFoundError:
        strucutres_not_found += 1
print(strucutres_not_found)

100


In [9]:
uiids = []
structures = []
cells = []
volumes = []
relevant_dimension_of_highest_dimensionality_substructures = []
binding_energy_per_substructures = []
binding_energies = []
labels = []
number_of_atoms_in_bulks = []
number_of_highest_dimensionality_substructures_in_bulks = []
sub_structure_spgs = []
sub_structure_dims = []
energy_bulks = []
sub_structure_formulas = []
for key in df2.keys():
    try:
        
        filename = uiid + '.xsf'
    
        with open(path + filename) as f:
            structure = ase.io.xsf.read_xsf(f, index=- 1, read_data=False)
        
        
        
        relevant_dimension_of_highest_dimensionality_substructures.append(df2[key]['binding_energy']['relevant_dimension_of_highest_dimensionality_substructure'])
        uiids.append(key)
        cells.append(df2[key]['cell'])
        volumes.append(df2[key]['volume'])
        binding_energy_per_substructures.append(df2[key]['binding_energy']['binding_energy_per_substructure'])
        tmp = ( -1000. *
                    df2[key]['binding_energy']['binding_energy_per_substructure']/
                    df2[key]['binding_energy']['relevant_dimension_of_highest_dimensionality_substructure'])
        binding_energies.append(tmp)
        
        if tmp <= 30:
            labels.append('EE')
        elif 30 < tmp <= 130:
            labels.append('PE')
        else:
            labels.append('HBE')
        
        structures.append(structure)        
        number_of_atoms_in_bulks.append(df2[key]['binding_energy']['number_of_atoms_in_bulk'])
        number_of_highest_dimensionality_substructures_in_bulks.append(df2[key]['binding_energy']['number_of_highest_dimensionality_substructures_in_bulk'])
        sub_structure_spgs.append(df2[key]['binding_energy']['sub_structure_spg'])
        sub_structure_dims.append(df2[key]['binding_energy']['sub_structure_dim'])
        energy_bulks.append(df2[key]['binding_energy']['energy_bulk'])
        sub_structure_formulas.append(df2[key]['binding_energy']['sub_structure_formulas'])
    except KeyError:
        continue

data_df2 = {
    'uiid' : uiids,
    'structure' : structures,
    'cell' : cells,
    'volume' : volumes,
    'relevant_dimension_of_highest_dimensionality_substructure' : relevant_dimension_of_highest_dimensionality_substructures,
    'binding_energy_per_substructure' : binding_energy_per_substructures,
    'binding_energy' : binding_energies,
    'label' : labels,
    'number_of_atoms_in_bulk' : number_of_atoms_in_bulks,
    'number_of_highest_dimensionality_substructures_in_bulk' : number_of_highest_dimensionality_substructures_in_bulks,
    'sub_structure_spg' : sub_structure_spgs,
    'sub_structure_dim' : sub_structure_dims,
    'energy_bulk' : energy_bulks,
    'sub_structure_formula' : sub_structure_formulas
}

dataframe_df2 = pd.DataFrame(data_df2)

In [10]:
dataframe_df2.head()

,uiid,structure,cell,volume,relevant_dimension_of_highest_dimensionality_substructure,binding_energy_per_substructure,binding_energy,label,number_of_atoms_in_bulk,number_of_highest_dimensionality_substructures_in_bulk,sub_structure_spg,sub_structure_dim,energy_bulk,sub_structure_formula
0,dec2d070-d2ad-41f3-b742-786368ade33a,"(Atom('K', [5.857725, 3.48476348, 3.73558257],...","[[2.1044124705, 1.2149831063, 34.279850741], [...",525.885600,15.340954,-0.165000,10.755509,EE,15,5,"[C2/m (12), C2/m (12), C2/m (12), C2/m (12), C...","[2, 2, 2, 2, 2]",-17198.871453,"[CdI2, CdI2, CdI2, CdI2, CdI2]"
1,51e630e8-d94f-421c-bd1e-3e2f298e740c,"(Atom('K', [5.857725, 3.48476348, 3.73558257],...","[[3.35814166734363, 3.21879556184565e-14, -3.2...",52.188194,9.766268,-0.149888,15.347484,EE,3,1,[P-3m1 (164)],[2],-5361.474444,[Cl2Fe]
2,527e7b32-d211-4743-9631-120122819190,"(Atom('K', [5.857725, 3.48476348, 3.73558257],...","[[2.14789490163679, -12.5878501270869, 0.0], [...",232.520056,18.471785,-0.454048,24.580650,EE,8,1,[P2_1/m (11)],[2],-26090.090963,[Er2Te6]
3,37bf14b1-6529-436c-a519-e1ad9b208784,"(Atom('K', [5.857725, 3.48476348, 3.73558257],...","[[-2.0745739339, 2.0745739339, 7.4278571834], ...",127.873741,17.215428,-0.747937,43.445746,PE,3,1,[C2/m (12)],[2],-2866.173187,[Ba2Hg]
4,e7a9cec3-f11a-4d16-a8c5-ebd758bb6592,"(Atom('K', [5.857725, 3.48476348, 3.73558257],...","[[4.6829418165, 0.0, 0.0], [0.0, 4.6829418165,...",166.823529,21.929944,-0.532853,24.297979,EE,6,1,[P4/nmm (129)],[2],-3217.539658,[Ba2H2I2]


In [11]:
from matminer.featurizers.conversions import ASEAtomstoStructure

In [12]:
aa2s = ASEAtomstoStructure()
dataframe_df2 = aa2s.fit_featurize_dataframe(dataframe_df2, col_id= 'structure', ignore_errors=True)

ASEAtomstoStructure:   0%|          | 0/2492 [00:00<?, ?it/s]

In [13]:
dataframe_df2.columns

Index(['uiid', 'structure', 'cell', 'volume',
       'relevant_dimension_of_highest_dimensionality_substructure',
       'binding_energy_per_substructure', 'binding_energy', 'label',
       'number_of_atoms_in_bulk',
       'number_of_highest_dimensionality_substructures_in_bulk',
       'sub_structure_spg', 'sub_structure_dim', 'energy_bulk',
       'sub_structure_formula', 'PMG Structure from ASE Atoms'],
      dtype='object')

In [14]:
structures = list(dataframe_df2["PMG Structure from ASE Atoms"])
gridpoints = list(dataframe_df2.index)

In [17]:
features = {n:{} for n in gridpoints}
for name in gridpoints:
    features[name]["structure"]=dataframe_df2["PMG Structure from ASE Atoms"][name]

In [18]:
data=pd.DataFrame.from_dict(features).T
data.head()

,structure
0,"[[5.857725 3.48476348 3.73558257] K, [ 1.952..."
1,"[[5.857725 3.48476348 3.73558257] K, [ 1.952..."
2,"[[5.857725 3.48476348 3.73558257] K, [ 1.952..."
3,"[[5.857725 3.48476348 3.73558257] K, [ 1.952..."
4,"[[5.857725 3.48476348 3.73558257] K, [ 1.952..."


In [50]:
features_dict={n:{} for n in gridpoints}
features_dict

{0: {},
 1: {},
 2: {},
 3: {},
 4: {},
 5: {},
 6: {},
 7: {},
 8: {},
 9: {},
 10: {},
 11: {},
 12: {},
 13: {},
 14: {},
 15: {},
 16: {},
 17: {},
 18: {},
 19: {},
 20: {},
 21: {},
 22: {},
 23: {},
 24: {},
 25: {},
 26: {},
 27: {},
 28: {},
 29: {},
 30: {},
 31: {},
 32: {},
 33: {},
 34: {},
 35: {},
 36: {},
 37: {},
 38: {},
 39: {},
 40: {},
 41: {},
 42: {},
 43: {},
 44: {},
 45: {},
 46: {},
 47: {},
 48: {},
 49: {},
 50: {},
 51: {},
 52: {},
 53: {},
 54: {},
 55: {},
 56: {},
 57: {},
 58: {},
 59: {},
 60: {},
 61: {},
 62: {},
 63: {},
 64: {},
 65: {},
 66: {},
 67: {},
 68: {},
 69: {},
 70: {},
 71: {},
 72: {},
 73: {},
 74: {},
 75: {},
 76: {},
 77: {},
 78: {},
 79: {},
 80: {},
 81: {},
 82: {},
 83: {},
 84: {},
 85: {},
 86: {},
 87: {},
 88: {},
 89: {},
 90: {},
 91: {},
 92: {},
 93: {},
 94: {},
 95: {},
 96: {},
 97: {},
 98: {},
 99: {},
 100: {},
 101: {},
 102: {},
 103: {},
 104: {},
 105: {},
 106: {},
 107: {},
 108: {},
 109: {},
 110: {},


In [51]:
from matminer.featurizers.site import CrystalNNFingerprint, GaussianSymmFunc, SiteElementalProperty,AGNIFingerprints,LocalPropertyDifference

In [52]:
## 2. Site Elemental Property
property_list=("Number","AtomicWeight","Row","Column","Electronegativity","CovalentRadius")
SEP = SiteElementalProperty(properties=property_list)
colnames=SEP._generate_column_labels(multiindex=False,return_errors=False)
for index,row in data.iterrows():
    structure=row["structure"]
    for atomidx in range(structure.num_sites):
        feat=SEP.featurize(structure,idx=atomidx)
        colname=["%s_%i"%(i,atomidx) for i in colnames]
        features_dict[index].update(dict(zip(colname, feat)))

In [53]:
features_dict

{0: {'site Number_0': 19.0,
  'site AtomicWeight_0': 39.0983,
  'site Row_0': 4.0,
  'site Column_0': 1.0,
  'site Electronegativity_0': 0.82,
  'site CovalentRadius_0': 203.0,
  'site Number_1': 19.0,
  'site AtomicWeight_1': 39.0983,
  'site Row_1': 4.0,
  'site Column_1': 1.0,
  'site Electronegativity_1': 0.82,
  'site CovalentRadius_1': 203.0,
  'site Number_2': 73.0,
  'site AtomicWeight_2': 180.94788,
  'site Row_2': 6.0,
  'site Column_2': 5.0,
  'site Electronegativity_2': 1.5,
  'site CovalentRadius_2': 170.0,
  'site Number_3': 73.0,
  'site AtomicWeight_3': 180.94788,
  'site Row_3': 6.0,
  'site Column_3': 5.0,
  'site Electronegativity_3': 1.5,
  'site CovalentRadius_3': 170.0,
  'site Number_4': 47.0,
  'site AtomicWeight_4': 107.8682,
  'site Row_4': 5.0,
  'site Column_4': 11.0,
  'site Electronegativity_4': 1.93,
  'site CovalentRadius_4': 145.0,
  'site Number_5': 47.0,
  'site AtomicWeight_5': 107.8682,
  'site Row_5': 5.0,
  'site Column_5': 11.0,
  'site Electrone

In [54]:
## 3. AGNI
property_list=("Number","AtomicWeight","Row","Column","Electronegativity","CovalentRadius")
AGNI = AGNIFingerprints(cutoff=5,directions=[None])
colnames=AGNI._generate_column_labels(multiindex=False,return_errors=False)
for index,row in data.iterrows():
    structure=row["structure"]
    for atomidx in range(structure.num_sites):
        feat=AGNI.featurize(structure,idx=atomidx)
        colname=["%s_%i"%(i,atomidx) for i in colnames]
        features_dict[index].update(dict(zip(colname, feat)))

In [56]:
features_dict[0]

{'site Number_0': 19.0,
 'site AtomicWeight_0': 39.0983,
 'site Row_0': 4.0,
 'site Column_0': 1.0,
 'site Electronegativity_0': 0.82,
 'site CovalentRadius_0': 203.0,
 'site Number_1': 19.0,
 'site AtomicWeight_1': 39.0983,
 'site Row_1': 4.0,
 'site Column_1': 1.0,
 'site Electronegativity_1': 0.82,
 'site CovalentRadius_1': 203.0,
 'site Number_2': 73.0,
 'site AtomicWeight_2': 180.94788,
 'site Row_2': 6.0,
 'site Column_2': 5.0,
 'site Electronegativity_2': 1.5,
 'site CovalentRadius_2': 170.0,
 'site Number_3': 73.0,
 'site AtomicWeight_3': 180.94788,
 'site Row_3': 6.0,
 'site Column_3': 5.0,
 'site Electronegativity_3': 1.5,
 'site CovalentRadius_3': 170.0,
 'site Number_4': 47.0,
 'site AtomicWeight_4': 107.8682,
 'site Row_4': 5.0,
 'site Column_4': 11.0,
 'site Electronegativity_4': 1.93,
 'site CovalentRadius_4': 145.0,
 'site Number_5': 47.0,
 'site AtomicWeight_5': 107.8682,
 'site Row_5': 5.0,
 'site Column_5': 11.0,
 'site Electronegativity_5': 1.93,
 'site CovalentRadi

In [57]:
## 4. Gaussian Symmetry Functions 
GSF = GaussianSymmFunc(cutoff=5)
colnames=GSF._generate_column_labels(multiindex=False,return_errors=False)
for index,row in data.iterrows():
    structure=row["structure"]
    for atomidx in range(structure.num_sites):
        feat=GSF.featurize(structure,idx=atomidx)
        colname=["%s_%i"%(i,atomidx) for i in colnames]
        features_dict[index].update(dict(zip(colname, feat)))

In [58]:
features_dict[0]

{'site Number_0': 19.0,
 'site AtomicWeight_0': 39.0983,
 'site Row_0': 4.0,
 'site Column_0': 1.0,
 'site Electronegativity_0': 0.82,
 'site CovalentRadius_0': 203.0,
 'site Number_1': 19.0,
 'site AtomicWeight_1': 39.0983,
 'site Row_1': 4.0,
 'site Column_1': 1.0,
 'site Electronegativity_1': 0.82,
 'site CovalentRadius_1': 203.0,
 'site Number_2': 73.0,
 'site AtomicWeight_2': 180.94788,
 'site Row_2': 6.0,
 'site Column_2': 5.0,
 'site Electronegativity_2': 1.5,
 'site CovalentRadius_2': 170.0,
 'site Number_3': 73.0,
 'site AtomicWeight_3': 180.94788,
 'site Row_3': 6.0,
 'site Column_3': 5.0,
 'site Electronegativity_3': 1.5,
 'site CovalentRadius_3': 170.0,
 'site Number_4': 47.0,
 'site AtomicWeight_4': 107.8682,
 'site Row_4': 5.0,
 'site Column_4': 11.0,
 'site Electronegativity_4': 1.93,
 'site CovalentRadius_4': 145.0,
 'site Number_5': 47.0,
 'site AtomicWeight_5': 107.8682,
 'site Row_5': 5.0,
 'site Column_5': 11.0,
 'site Electronegativity_5': 1.93,
 'site CovalentRadi

In [59]:
df_features=pd.DataFrame.from_dict(features_dict).T.astype(float)

In [61]:
df_features.columns

Index(['site Number_0', 'site AtomicWeight_0', 'site Row_0', 'site Column_0',
       'site Electronegativity_0', 'site CovalentRadius_0', 'site Number_1',
       'site AtomicWeight_1', 'site Row_1', 'site Column_1',
       ...
       'G4_0.005_4.0_1.0_14', 'G4_0.005_4.0_-1.0_14', 'G2_0.05_15',
       'G2_4.0_15', 'G2_20.0_15', 'G2_80.0_15', 'G4_0.005_1.0_1.0_15',
       'G4_0.005_1.0_-1.0_15', 'G4_0.005_4.0_1.0_15', 'G4_0.005_4.0_-1.0_15'],
      dtype='object', length=352)

In [62]:
df_features.to_pickle("features_assigned_different_columns.pkl")